<a href="https://colab.research.google.com/github/vilsonrodrigues/PerseuProject/blob/master/Montando_a_tabela_com_os_arquivos_do_sigaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-processamento

Importando bibliotecas

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime

Lendo as url's das notas

In [0]:
notas_url = [
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2015.1.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2015.2.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2016.1.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2016.2.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.1.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.2.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.1.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.2.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.1.csv",
             "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.2.csv"
            ]
carga_horaria = "http://dados.ufrn.br/dataset/3fea67e8-6916-4ed0-aaa6-9a8ca06a9bdc/resource/9a3521d2-4bc5-4fda-93f0-f701c8a20727/download/componentes-curriculares-presenciais.csv"

Lendo como CSV

In [0]:
col = ["periodo","codigo","nome","turma","siape","horario","matricula","nota","situacao"]
notas2015_1 = pd.read_csv(notas_url[0], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2015_2 = pd.read_csv(notas_url[1], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2016_1 = pd.read_csv(notas_url[2], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2016_2 = pd.read_csv(notas_url[3], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2017_1 = pd.read_csv(notas_url[4], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2017_2 = pd.read_csv(notas_url[5], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2018_1 = pd.read_csv(notas_url[6], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2018_2 = pd.read_csv(notas_url[7], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2019_1 = pd.read_csv(notas_url[8], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
#notas2019_2 = pd.read_csv(notas_url[9], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)

carga = pd.read_csv(carga_horaria,sep=";")
carga = carga[["codigo","ch_total"]]

In [0]:
notas2015_1.sample(10)

,periodo,codigo,nome,turma,siape,horario,matricula,nota,situacao
33430,1,ECT1203,LINGUAGEM DE PROGRAMAÇÃO,03D,3687943-8,7T12 46N34 (02/02/2015 - 20/06/2015),2014026782,-,EXCLUIDA
28963,1,ECT1211,ÁLGEBRA LINEAR,06,2998728-0,235T56 (02/02/2015 - 05/06/2015),2012925918,-,CANCELADO
10340,1,ECT2414,ELETRICIDADE APLICADA,03E,"1658061-5,2008861-4,2223220-8","35N34 (02/02/2015 - 31/03/2015), 7T12 35N34 ...",2012037858,-,EXCLUIDA
43606,1,ECT2203,LÓGICA DE PROGRAMAÇÃO,03B,1805797-9,7M12 46N34 (02/02/2015 - 06/06/2015),2013090395,0.80,REPROVADO
2678,1,ECT2203,LÓGICA DE PROGRAMAÇÃO,02C,"1060782-1,1434796-4",6M56 46T56 (02/02/2015 - 06/06/2015),2014080277,-,EXCLUIDA
3164,1,ECT2104,QUÍMICA GERAL,01B,3432383-1,35M56 3T12 (02/02/2015 - 20/06/2015),2013077606,-,EXCLUIDA
41880,1,ECT1405,MODELAGEM INTEGRADA,01A,"2066958-7,2571613-3",3M12 346T34 (02/02/2015 - 12/04/2015),2012922596,0.20,REPROVADO
30874,1,ECT2412,MECÂNICA DOS SÓLIDOS,02,2212882-6,25T34 (02/02/2015 - 20/06/2015),2013070884,6.40,APROVADO POR NOTA
35907,1,ECT2204,INTRODUÇÃO À FÍSICA CLÁSSICA I,03,1717472-6,36T12 (02/02/2015 - 20/06/2015),2013021754,-,EXCLUIDA
30094,1,MEC1834,INTRODUÇÃO AOS ELEMENTOS DE MÁQUINAS,01,5220696-5,46M56,2012922700,8.10,APROVADO


Função para converter para string

In [0]:
def converteParaString(table):
  table["codigo"]     = table["codigo"].astype(str)
  table["disciplina"] = table["disciplina"].astype(str)
  table["turma"]      = table["turma"].astype(str)
  table["horario"]    = table["horario"].astype(str)
  #table["matricula"]  = table["matricula"].astype(str)
  table["situacao"]   = table["situacao"].astype(str)
  return table

Função para converter todos os reprovados e aprovados diferentes para Reprovado e Aprovado

In [0]:
def fazerReplace(table):
  table.loc[table["situacao"].str.contains("REPROVADO"),"situacao"] = "REPROVADO"
  table.loc[table["situacao"].str.contains("APROVADO"),"situacao"] = "APROVADO"
  return table

Função filtro para pegar apenas as situações onde o aluno conseguiu se matricular na disciplina

In [0]:
def filtroSituacoesValidas(table):
  table = table.loc[ (table["situacao"].str.contains("APROVADO")) |
                     (table["situacao"].str.contains("REPROVADO")) 
                   ] 
  #tem disciplina que só contam para a horas complementares .. não tem nota
  table = table.loc[ ~(table["nota"].str.contains("-")) ]
  table["nota"]   = table["nota"].astype(float)
  
  return table

Função para montar a tabela com os antecedentes dos alunos

In [0]:
def criaDataFrameAntecedentes(table):
  table = table.merge(carga,on="codigo")
  
  df = pd.DataFrame()
  final = pd.DataFrame()

  df = table.groupby("matricula")["nota"].mean()
  df = df.reset_index()
  df.rename(columns={"nota":"media"},inplace=True)
  final = df

  df = table.groupby("matricula")["nota"].min()
  df = df.reset_index()
  df.rename(columns={"nota":"min"},inplace=True)
  final = final.merge(df,on="matricula", how ='outer')

  df = table.groupby("matricula")["nota"].max()
  df = df.reset_index()
  df.rename(columns={"nota":"max"},inplace=True)
  final = final.merge(df,on="matricula",how ='outer')

  df = table.groupby("matricula")["codigo"].count()
  df = df.reset_index()
  df.rename(columns={"codigo":"QuantDisciCursadas"},inplace=True)
  final = final.merge(df,on="matricula", how ='outer')

  df = table.groupby(["matricula","situacao"])["codigo"].count()
  df = df.reset_index()
  df.rename(columns={"codigo":"quanSituacao"},inplace=True)

  aprovado = df.loc[df["situacao"] == "APROVADO",:]
  aprovado.rename(columns={"quanSituacao":"quantAprovado"},inplace=True)
  final = final.merge(aprovado,on="matricula", how ='outer')
  
  reprovado = df.loc[df["situacao"] == "REPROVADO",:]
  reprovado.rename(columns={"quanSituacao":"quantReprovado"},inplace=True)
  final = final.merge(reprovado,on="matricula", how ='outer')
  final = final.drop(["situacao_x","situacao_y"],axis=1)
 
  df = table.groupby("matricula")["ch_total"].sum()
  df = df.reset_index()
  df.rename(columns={"ch_total":"carga_total"},inplace=True)
  final = final.merge(df,on="matricula", how ='outer')
  
  final["anoSemestre"] = table["ano"]
  return final

Função para inserir o ano de ingresso do aluno

In [0]:
def AnoIngresso(table) : 
  table["matricula"] = table["matricula"].astype(str)
  table["AnoIngresso"] = table["matricula"].str[0:4]
  table["AnoIngresso"] = table["AnoIngresso"].astype(float)
  table["anoSemestre"] = table["anoSemestre"].astype(float)
  table["AnosMatriculado"] = table["anoSemestre"] - table["AnoIngresso"]
  table["AnosMatriculado"] = table["AnosMatriculado"].astype(int)
  return table

Função para adicionar os anos a cada semestre

In [0]:
def concatena(table,i,ano):
  if i%2==0:
    ano += 1
  table = fazerReplace(table)  
  table["ano"] = str(ano)+"."+str(table["periodo"].iloc[0])
  table.rename(columns={"nome":"disciplina"},inplace=True)
  table.drop(columns=["periodo","siape"], inplace=True)
  table = filtroSituacoesValidas(table)
  table = converteParaString(table)
  notas = table
  table = criaDataFrameAntecedentes(table)
  table = table.fillna(0)
  table["quantAprovado"] = table["quantAprovado"].astype(int)
  table["quantReprovado"] = table["quantReprovado"].astype(int)
  table = AnoIngresso(table)
  i+=1
  return (table,i,ano,notas)

Criando 10 novos dataframes

In [0]:
semestre2015_1 = pd.DataFrame()
semestre2015_2 = pd.DataFrame()
semestre2016_1 = pd.DataFrame()
semestre2016_2 = pd.DataFrame()
semestre2017_1 = pd.DataFrame()
semestre2017_2 = pd.DataFrame()
semestre2018_1 = pd.DataFrame()
semestre2018_2 = pd.DataFrame()
semestre2019_1 = pd.DataFrame()
#semestre2019_2 = pd.DataFrame()

Chamada da função de inserir o ano + período, nela também é chamada a de filtro de situação e a de conversão para string

In [0]:
ano = 2014
i = 2

(semestre2015_1,i,ano,notas2015_1) = concatena(notas2015_1,i,ano)
(semestre2015_2,i,ano,notas2015_2) = concatena(notas2015_2,i,ano)
(semestre2016_1,i,ano,notas2016_1) = concatena(notas2016_1,i,ano)
(semestre2016_2,i,ano,notas2016_2) = concatena(notas2016_2,i,ano)
(semestre2017_1,i,ano,notas2017_1) = concatena(notas2017_1,i,ano)
(semestre2017_2,i,ano,notas2017_2) = concatena(notas2017_2,i,ano)
(semestre2018_1,i,ano,notas2018_1) = concatena(notas2018_1,i,ano)
(semestre2018_2,i,ano,notas2018_2) = concatena(notas2018_2,i,ano)
(semestre2019_1,i,ano,notas2019_1) = concatena(notas2019_1,i,ano)
#(semestre2019_2,i,ano,notas2019_2) = concatena(notas2019_2,i,ano)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [0]:
semestre2015_1

,matricula,media,min,max,QuantDisciCursadas,quantAprovado,quantReprovado,carga_total,anoSemestre,AnoIngresso,AnosMatriculado
0,2009028341,4.971429,0.7,9.1,7,5,2,450,2015.1,2009.0,6
1,2009028457,3.950000,0.0,6.1,4,2,2,210,2015.1,2009.0,6
2,2009028473,6.485714,5.2,8.6,7,7,0,420,2015.1,2009.0,6
3,2009029488,4.033333,0.0,7.2,6,3,3,360,2015.1,2009.0,6
4,2009029534,5.880000,2.4,7.1,5,4,1,270,2015.1,2009.0,6
...,...,...,...,...,...,...,...,...,...,...,...
3069,2015096155,3.883333,1.2,6.6,6,3,3,330,2015.1,2015.0,0
3070,2015096217,4.733333,0.2,7.6,6,3,3,330,2015.1,2015.0,0
3071,2015096315,3.050000,0.0,6.8,6,2,4,330,2015.1,2015.0,0
3072,2015096398,6.050000,5.4,6.7,2,2,0,120,2015.1,2015.0,0


# Projeto Perseu - análise de solicitação de matricula

Media de todas as disciplinas

Maior nota

Menor nota

Quantidade de disciplinas

Carga horaria total

Quantidade de aprovação e reprovação

In [0]:
semestre2017_1.head(2)

,matricula,media,min,max,QuantDisciCursadas,quantAprovado,quantReprovado,carga_total,anoSemestre,AnoIngresso,AnosMatriculado
0,2009028457,3.05,0.0,6.8,4,2,2,270,2017.1,2009.0,8
1,2009030796,5.60,5.5,5.7,2,1,1,120,2017.1,2009.0,8


In [0]:
notas2017_2.head(2)

,codigo,disciplina,turma,horario,matricula,nota,situacao,ano
12,ECT2101,PRÉ-CÁLCULO,01,2356T12 (24/07/2017 - 30/09/2017),20170148556,6.3,REPROVADO,2017.2
13,ECT2102,VETORES E GEOMETRIA ANALÍTICA,01,36T34 (24/07/2017 - 16/12/2017),20170148556,1.6,REPROVADO,2017.2


Função para concatenar o antecedente com a situação em matéria posterior

In [0]:
def concatenaAntecessoresComResultadoPosterior(tableAntecedente,tableAtual,situacaoPosterior,final):
  
  tableAntecedente["matricula"] = tableAntecedente["matricula"].astype(int)
  tableAtual["matricula"] = tableAtual["matricula"].astype(int)
  disc =  situacaoPosterior[["codigo", "matricula","situacao"]]

  tableAtual = tableAtual.iloc[:,[0,7,4,-1]]
  tableAtual.rename(columns={"carga_total":"carga_total_atual","QuantDisciCursadas":"QuantDisciAtual"}, inplace=True) 
  #tableAntecedente = tableAntecedente[["matricula","media","min","max","QuantDisciCursadas","quantAprovado","quantReprovado","carga_total"]]
  tableAntecedente = tableAntecedente.iloc[:,:-3]

  tableAntecedente = pd.merge(tableAntecedente,disc,on="matricula")
  tableAtual = pd.merge(tableAntecedente,tableAtual,on="matricula")
  
  if(final.empty):
    final = tableAtual
  else:
    final = pd.concat([final,tableAtual],ignore_index=True,axis=0)
 
  return final

Chamada das funções onde armazena em dfFinal

In [0]:
dfFinal = pd.DataFrame()

dfFinal = concatenaAntecessoresComResultadoPosterior(semestre2015_1,semestre2015_2,notas2015_2,dfFinal)
dfFinal = concatenaAntecessoresComResultadoPosterior(semestre2015_2,semestre2016_1,notas2016_1,dfFinal)
dfFinal = concatenaAntecessoresComResultadoPosterior(semestre2016_1,semestre2016_2,notas2016_2,dfFinal)
dfFinal = concatenaAntecessoresComResultadoPosterior(semestre2016_2,semestre2017_1,notas2017_1,dfFinal)
dfFinal = concatenaAntecessoresComResultadoPosterior(semestre2017_1,semestre2017_2,notas2017_2,dfFinal)
dfFinal = concatenaAntecessoresComResultadoPosterior(semestre2017_2,semestre2018_1,notas2018_1,dfFinal)
dfFinal = concatenaAntecessoresComResultadoPosterior(semestre2018_1,semestre2018_2,notas2018_2,dfFinal)
dfFinal = concatenaAntecessoresComResultadoPosterior(semestre2018_2,semestre2019_1,notas2019_1,dfFinal)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [0]:
dfFinal.head(5)

,matricula,media,min,max,QuantDisciCursadas,quantAprovado,quantReprovado,carga_total,codigo,situacao,carga_total_atual,QuantDisciAtual,AnosMatriculado
0,2009028457,3.95,0.0,6.1,4,2,2,210,ECT2411,REPROVADO,330,5,6
1,2009028457,3.95,0.0,6.1,4,2,2,210,ECT1569,APROVADO,330,5,6
2,2009028457,3.95,0.0,6.1,4,2,2,210,ECT2304,REPROVADO,330,5,6
3,2009028457,3.95,0.0,6.1,4,2,2,210,CIV1206,APROVADO,330,5,6
4,2009028457,3.95,0.0,6.1,4,2,2,210,ECT2303,REPROVADO,330,5,6


Selecionando as disciplinas da ECT

In [0]:
disciplinasECT = dfFinal.loc[dfFinal["codigo"].str.contains("ECT22") | dfFinal["codigo"].str.contains("ECT23") | dfFinal["codigo"].str.contains("ECT24"),:]
disciplinasECT.rename(columns={"codigo":"disciplina"},inplace=True)
disciplinasECT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52490 entries, 0 to 88984
Data columns (total 13 columns):
matricula             52490 non-null int64
media                 52490 non-null float64
min                   52490 non-null float64
max                   52490 non-null float64
QuantDisciCursadas    52490 non-null int64
quantAprovado         52490 non-null int64
quantReprovado        52490 non-null int64
carga_total           52490 non-null int64
disciplina            52490 non-null object
situacao              52490 non-null object
carga_total_atual     52490 non-null int64
QuantDisciAtual       52490 non-null int64
AnosMatriculado       52490 non-null int64
dtypes: float64(3), int64(8), object(2)
memory usage: 5.6+ MB


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


Contando quantas vezes o aluno já reprovou na matéria

In [0]:
prov = disciplinasECT.groupby(["matricula","disciplina"])["situacao"].count().reset_index()
disciplinasECT["vezesReprovado"] = prov["situacao"] #vari
disciplinasECT.replace(np.nan, 0, inplace=True)
disciplinasECT["vezesReprovado"] = disciplinasECT["vezesReprovado"].astype(int)
disciplinasECT.head()

In [0]:
disciplinasECT.loc[(disciplinasECT["matricula"]==2009028457),:].sort_values(by="disciplina")

,matricula,media,min,max,QuantDisciCursadas,quantAprovado,quantReprovado,carga_total,disciplina,situacao,carga_total_atual,QuantDisciAtual,AnosMatriculado
35470,2009028457,2.733333,0.0,5.6,3,1,2,180,ECT2206,APROVADO,270,4,8
11842,2009028457,3.340000,0.0,8.8,5,2,3,330,ECT2302,APROVADO,255,5,7
4,2009028457,3.950000,0.0,6.1,4,2,2,210,ECT2303,REPROVADO,330,5,6
35467,2009028457,2.733333,0.0,5.6,3,1,2,180,ECT2303,REPROVADO,270,4,8
2,2009028457,3.950000,0.0,6.1,4,2,2,210,ECT2304,REPROVADO,330,5,6
11843,2009028457,3.340000,0.0,8.8,5,2,3,330,ECT2304,REPROVADO,255,5,7
35468,2009028457,2.733333,0.0,5.6,3,1,2,180,ECT2304,REPROVADO,270,4,8
0,2009028457,3.950000,0.0,6.1,4,2,2,210,ECT2411,REPROVADO,330,5,6
11841,2009028457,3.340000,0.0,8.8,5,2,3,330,ECT2411,REPROVADO,255,5,7


Gerando CSV

In [0]:
disciplinasECT.to_csv(r"disciplinasECT201520191.csv")